In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
choices_21 = pd.read_excel('data/2021.xlsx', sheet_name='Resultaten')
# choices_20 = pd.read_excel('data/2020.xlsx', sheet_name='Resultaten')
# choices_19 = pd.read_excel('data/2019.xlsx', sheet_name='Resultaten')
# choices_18 = pd.read_excel('data/2018.xlsx', sheet_name='Resultaten')
# choices_17 = pd.read_excel('data/2017.xlsx', sheet_name='Resultaten')

In [8]:
schools_21 = pd.read_excel('data/2021.xlsx', sheet_name='Klassen')
schools_20 = pd.read_excel('data/2020.xlsx', sheet_name='Klassen')
schools_19 = pd.read_excel('data/2019.xlsx', sheet_name='Klassen')
schools_18 = pd.read_excel('data/2018.xlsx', sheet_name='Klassen')
schools_17 = pd.read_excel('data/2017.xlsx', sheet_name='Klassen')

In [9]:
choices_21.columns

Index(['Basisschool advies', 'Lotnummer', 'Geplaatst op', 'Positie',
       'Voorrang/Hardheid eerste voorkeur', 'Voorkeur 1', 'Voorkeur 2',
       'Voorkeur 3', 'Voorkeur 4', 'Voorkeur 5', 'Voorkeur 6', 'Voorkeur 7',
       'Voorkeur 8', 'Voorkeur 9', 'Voorkeur 10', 'Voorkeur 11', 'Voorkeur 12',
       'Voorkeur 13', 'Voorkeur 14', 'Voorkeur 15', 'Voorkeur 16',
       'Voorkeur 17', 'Voorkeur 18', 'Voorkeur 19', 'Voorkeur 20',
       'Voorkeur 21', 'Voorkeur 22', 'Voorkeur 23', 'Voorkeur 24',
       'Voorkeur 25', 'Voorkeur 26', 'Voorkeur 27', 'Voorkeur 28',
       'Voorkeur 29', 'Voorkeur 30', 'Voorkeur 31', 'Voorkeur 32',
       'Voorkeur 33', 'Voorkeur 34'],
      dtype='object')

In [13]:
choices = [choices_21, choices_20, choices_19, choices_18, choices_17]
for c in choices:
    print(c.shape)
    # print(c.columns)

(7978, 39)
(7672, 39)
(7580, 39)
(7420, 39)
(7638, 44)


- 2018 - 2021: each year students is allowed to choose maximum 34 schools; 2017: 39
- In reality, 2021: students only choose $\leq 22$ schools; 2020: $\leq 18$; 2019: $\leq 21$; 2018: $\leq 19$

In [ ]:
# total count and percentage of missing values per column
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(choices_17)

In [10]:
edutypepop = choices_21['Basisschool advies'].value_counts().rename_axis('edu_type').reset_index(name='count')
edutypepop

,edu_type,count
0,vwo,2290
1,havo/vwo,1239
2,havo,1206
3,vmbo-t,1126
4,vmbo-t/havo,797
5,vmbo-k,618
6,vmbo-b,390
7,vmbo-b/k,312


In [11]:
# import plotly.graph_objects as go

# labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
# values = [4500, 2500, 1053, 500]

# fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
#                              insidetextorientation='radial'
#                             )])
# fig.show()

import plotly.express as px

# df = px.data.gapminder().query("continent == 'Asia'")

fig = px.pie(edutypepop, values ='count', names='edu_type')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

fig.show()


# fig = px.pie(df, values='pop', names='country')
# fig.update_traces(textposition='inside')
# fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
# fig.show()



In [ ]:
# list of schools, adding 'None' (students leave it blank)
schools = list(schools_21['Key'])
schools.append('None')
len(schools)